# M3 Final Project

**Authors:**
- Hazel Bunning
- Marcel Grosjean

# 1. Project setup

## 1.1 Import the necessary libraries:

First, install the needed packages:

In [ ]:
!pip install openpyxl

Then import the libraries:

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import load_workbook


## 1.2 Download the source file

https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees.assetdetail.31425965.html

In [ ]:
# Url of the data file
source_file_url = f"https://dam-api.bfs.admin.ch/hub/api/dam/assets/31425965/master"
response = requests.get(source_file_url)
response.raise_for_status()

# Store the file locally
with open("source_file.xlsx", "wb") as file:
  file.write(response.content)

!ls

sample_data  source_file.xlsx


Parse the source file and save data to `df` dataset:

In [ ]:
# return the int value of each cell
def parse_cell(val: str | None) -> int:
  if val is None:
    return 0
  try:
    return int(val)
  except ValueError:
    return 0

# Create our empty dataset
headers = ["canton", "year", "male", "female", "-18", "18-24", "25+", "swiss", "permit", "asylium", "no_permit", "total"]
# df is our final dataset that contains all the data
df = pd.DataFrame(columns=headers)


# read the source file and get all the tabs
workbook = load_workbook(filename="source_file.xlsx")
tab_names = workbook.sheetnames
# read sheet by sheet
for tab_name in tab_names:
  sheet = workbook[tab_name]

  # read line by line (canton by canton)
  for row in range(0, 26):
    i = row + 6 # data start at line 6

    new_row = {
      "canton": sheet[f"A{i}"].value,
      "year": parse_cell(tab_name),
      "male": parse_cell(sheet[f"C{i}"].value),
      "female": parse_cell(sheet[f"D{i}"].value),
      "-18": parse_cell(sheet[f"E{i}"].value),
      "18-24": parse_cell(sheet[f"F{i}"].value),
      "25+": parse_cell(sheet[f"G{i}"].value),
      "swiss": parse_cell(sheet[f"H{i}"].value),
      "permit": parse_cell(sheet[f"I{i}"].value),
      "asylium": parse_cell(sheet[f"J{i}"].value),
      "no_permit": parse_cell(sheet[f"K{i}"].value),
      "total": parse_cell(sheet[f"B{i}"].value),
    }

    # add the new row to the pd dataset
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df

,canton,year,male,female,-18,18-24,25+,swiss,permit,asylium,no_permit,total
0,Zurich,2024,370,42,5,72,335,115,105,59,133,412
1,Bern,2024,219,8,13,25,189,46,41,9,131,227
2,Lucerne,2024,33,2,0,11,24,8,12,10,5,35
3,Uri,2024,0,0,0,0,0,0,0,0,0,0
4,Schwyz,2024,9,1,0,0,10,2,3,1,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...
957,Vaud,1988,157,18,67,108,69,53,53,0,0,175
958,Valais,1988,49,0,7,42,28,10,11,0,0,49
959,Neuchâtel,1988,16,2,8,10,8,8,2,0,0,18
960,Geneva,1988,205,25,58,172,77,51,102,0,0,230


In [ ]:
# the features "X" include canton, year, male, female, -18, 18-24, 25+, swiss, permit, asylium, no_permit
X = df.iloc[:, :-1].values # get all the rows and get all the column except the last one
X

array([['Zurich', 2024, 370, ..., 105, 59, 133],
       ['Bern', 2024, 219, ..., 41, 9, 131],
       ['Lucerne', 2024, 33, ..., 12, 10, 5],
       ...,
       ['Neuchâtel', 1988, 16, ..., 2, 0, 0],
       ['Geneva', 1988, 205, ..., 102, 0, 0],
       ['Jura', 1988, 6, ..., 1, 0, 0]], dtype=object)

In [ ]:
# the dependent variable "y" is the total people in detention per year
y = df.iloc[:, -1].values # get all the rows and get only the last column
y

array([412, 227, 35, 0, 10, 3, 6, 5, 12, 61, 50, 72, 32, 18, 4, 1, 41, 6,
       93, 38, 100, 352, 106, 46, 326, 20, 382, 217, 29, 0, 13, 3, 5, 8,
       8, 43, 52, 78, 39, 13, 6, 1, 50, 7, 76, 24, 74, 333, 85, 40, 322,
       16, 352, 222, 28, 0, 9, 4, 1, 3, 8, 48, 33, 54, 43, 16, 4, 2, 54,
       9, 87, 13, 65, 356, 86, 41, 321, 13, 374, 212, 27, 0, 7, 2, 5, 3,
       9, 48, 39, 55, 23, 33, 4, 0, 36, 5, 76, 26, 85, 312, 90, 35, 449,
       13, 377, 203, 38, 0, 6, 5, 6, 7, 12, 30, 54, 65, 5, 33, 1, 0, 55,
       6, 78, 27, 63, 317, 92, 32, 377, 13, 335, 196, 33, 0, 8, 4, 8, 2,
       13, 44, 34, 73, 36, 12, 2, 2, 47, 5, 75, 22, 63, 266, 102, 46, 424,
       15, 303, 195, 37, 0, 9, 3, 6, 3, 10, 44, 38, 80, 46, 16, 4, 1, 61,
       5, 87, 23, 74, 254, 91, 45, 394, 14, 287, 208, 35, 0, 5, 3, 8, 4,
       10, 45, 30, 85, 48, 22, 1, 0, 51, 5, 74, 29, 59, 243, 92, 41, 280,
       8, 337, 229, 29, 0, 8, 2, 2, 1, 4, 49, 32, 87, 36, 19, 2, 2, 60, 1,
       91, 27, 62, 209, 105, 39, 296, 16, 36

# 1.3 No missing data

#2. Encoding Categorical Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Assuming df is already defined as in the previous context
# Extract features X and dependent variable y
X = df.iloc[:, :-1].values  # Extract all columns except the last one
y = df.iloc[:, -1].values    # Extract the last column as the target variable

# Print the shape of X to confirm it's not empty
print("Original shape of X:", X.shape)
print("First 5 rows of X:\n", X[:5])  # Display first 5 rows to verify

# Define categorical feature indices (assuming 'canton' is at index 0 and 'year' is at index 1)
categorical_indices = [0, 1]  # Adjust these indices according to your feature order

# Create the ColumnTransformer for one-hot encoding
ct = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(), categorical_indices)
    ],
    remainder='passthrough'  # Keep the rest of the columns unchanged
)

# Transform the features
try:
    X_encoded = ct.fit_transform(X)  # Transform X using the ColumnTransformer
    print("Transformation successful.")
except Exception as e:
    print(f"Error during transformation: {e}")

# Print the shape of X_encoded to confirm it is transformed
print("Shape of encoded features (before conversion):", X_encoded.shape)

# Convert to a dense NumPy array
X_encoded = X_encoded.toarray()  # Correctly convert sparse matrix to a dense array

# Display the shape and the first few rows for verification
print("Shape of encoded features (after conversion):", X_encoded.shape)
print("First 5 rows of encoded features:\n", X_encoded[:5])  # Display first 5 rows


Original shape of X: (962, 11)
First 5 rows of X:
 [['Zurich' 2024 370 42 5 72 335 115 105 59 133]
 ['Bern' 2024 219 8 13 25 189 46 41 9 131]
 ['Lucerne' 2024 33 2 0 11 24 8 12 10 5]
 ['Uri' 2024 0 0 0 0 0 0 0 0 0]
 ['Schwyz' 2024 9 1 0 0 10 2 3 1 4]]
Transformation successful.
Shape of encoded features (before conversion): (962, 72)
Shape of encoded features (after conversion): (962, 72)
First 5 rows of encoded features:
 [[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   1. 370.  42.   5.  72. 335. 115. 105.
   59. 133.]
 [  0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.

#2.1 Splitting the dataset into Training set and Test set

In [ ]:
# To split the dataset, we use the train_test_split function from scikit-learn’s model_selection module.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train

array([['Zurich', 2006, 425, ..., 75, 41, 236],
       ['Jura', 1992, 9, ..., 1, 0, 0],
       ['Uri', 2003, 0, ..., 0, 0, 0],
       ...,
       ['Lucerne', 1991, 19, ..., 8, 0, 0],
       ['Thurgau', 2008, 24, ..., 8, 3, 3],
       ['Geneva', 2021, 414, ..., 79, 0, 319]], dtype=object)

In [ ]:
X_test

array([[0.0, 0.0, 0.0, ..., 1, 0, 3],
       [0.0, 0.0, 0.0, ..., 1, 0, 4],
       [0.0, 0.0, 0.0, ..., 3, 3, 7],
       ...,
       [0.0, 0.0, 0.0, ..., 0, 1, 2],
       [0.0, 1.0, 0.0, ..., 0, 0, 0],
       [0.0, 0.0, 0.0, ..., 1, 0, 0]], dtype=object)

In [ ]:
y_train

array([454, 9, 0, 39, 5, 13, 23, 327, 7, 1, 0, 246, 4, 6, 19, 35, 35, 1,
       0, 478, 337, 3, 28, 10, 1, 191, 2, 26, 3, 92, 1, 5, 33, 0, 1, 25,
       20, 5, 50, 80, 1, 3, 4, 3, 0, 1, 27, 67, 41, 0, 0, 4, 44, 31, 342,
       1, 1, 0, 58, 33, 67, 454, 25, 2, 28, 0, 7, 1, 86, 27, 69, 6, 62, 1,
       5, 50, 35, 207, 323, 5, 15, 267, 7, 450, 18, 44, 91, 37, 8, 7, 181,
       23, 84, 9, 2, 270, 62, 0, 27, 8, 212, 75, 24, 5, 16, 44, 34, 0, 10,
       22, 206, 412, 24, 0, 421, 2, 7, 38, 61, 317, 242, 475, 35, 23, 56,
       14, 3, 228, 9, 4, 196, 1, 77, 2, 84, 4, 10, 29, 107, 85, 16, 96,
       16, 0, 34, 5, 6, 85, 3, 356, 0, 166, 6, 2, 0, 363, 36, 3, 4, 33,
       46, 4, 0, 1, 7, 6, 62, 9, 74, 0, 15, 326, 3, 160, 13, 4, 434, 16,
       18, 379, 2, 10, 23, 62, 72, 0, 93, 87, 257, 31, 0, 2, 48, 18, 34,
       17, 70, 35, 79, 119, 32, 287, 8, 6, 85, 2, 37, 13, 17, 3, 35, 21,
       74, 31, 1, 303, 3, 18, 3, 1, 96, 51, 81, 58, 86, 30, 8, 55, 464,
       377, 54, 154, 7, 19, 4, 28, 55, 412, 3,

In [ ]:
y_test

array([6, 7, 16, 189, 5, 5, 1, 29, 9, 373, 68, 2, 32, 175, 54, 25, 13, 3,
       4, 243, 46, 55, 445, 11, 15, 46, 39, 36, 7, 87, 184, 0, 90, 44, 1,
       230, 12, 3, 46, 2, 24, 13, 188, 0, 37, 76, 2, 8, 11, 28, 29, 36,
       11, 16, 413, 0, 44, 0, 44, 321, 191, 214, 45, 87, 27, 13, 54, 14,
       25, 4, 89, 36, 27, 103, 95, 0, 39, 35, 229, 1, 93, 5, 506, 6, 244,
       50, 5, 551, 76, 55, 87, 34, 25, 149, 399, 420, 132, 65, 2, 76, 10,
       51, 0, 239, 8, 7, 3, 35, 8, 10, 43, 8, 63, 46, 7, 101, 5, 44, 81,
       43, 12, 0, 8, 0, 23, 3, 37, 4, 8, 3, 63, 0, 89, 8, 69, 57, 6, 8,
       416, 273, 235, 39, 30, 59, 8, 180, 6, 38, 2, 105, 4, 0, 54, 5, 83,
       80, 44, 9, 10, 374, 16, 43, 0, 23, 26, 18, 7, 16, 73, 5, 324, 14,
       0, 9, 84, 29, 0, 324, 40, 0, 2, 324, 13, 45, 43, 23, 44, 0, 3, 8,
       3, 1, 1], dtype=object)

# 2.2 Feature Scaling